In [2]:
from IPython.core.display import HTML
display(HTML("<style>.container { width:95% !important; }</style>"))   
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Author: Andrew Tarzia

Date Created: 12 Feb 2018

License:

# Notebook for screening BRENDA database post pI screening

TODO:
- take BRENDA files and split into property specific files using code in python script
- filter out proteins based on BRENDA information
- collect product and substrate names
- output
- turn notebook into presentation of process for an example.
- move to scripts
    - generalise to running in current working dir
    - add license

In [3]:
import os
import pandas as pd
import glob
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import time
import numpy as np
from shutil import copyfile
%aimport parameters
%aimport brenda_IO

# Notes:


In [4]:
# get parameters
param_dict = parameters.get_parameters()

# input parameters
database_directory = '/home/atarzia/psp/sequence_db/test_dataset/'
# database_directory = '/home/atarzia/psp/sequence_db/linb_dataset/'

output_dir = database_directory

# get input FASTA file names
database_names = []
for i in glob.glob("*fasta"):
    if "_mod" not in i:
        database_names.append(i)

# output CSV file
redo_pi = True
if redo_pi is True:
    string = ''
    for i in param_dict['out_columns_br']:
        if i == param_dict['out_columns_br'][-1]:
            string += i
        else:
            string += i+','
    string += '\n'
    with open(output_dir+param_dict['out_CSV_br'], 'w') as f:
        f.write(string)

In [11]:
print('=====================================================================')
print('                     Screening enzymes with Brenda!                  ')
print('=====================================================================')
# read in pI output file
pi_data = pd.read_csv(output_dir+param_dict['out_CSV_pi'], index_col=False)
# only keep those that pass the pI test
pi_passed_data = pi_data[pi_data['pi'] < param_dict['cutoff_pi']]
print('--- ', 
      len(pi_passed_data), 'proteins out of', len(pi_data), 
      'remain, OR', int(len(pi_passed_data)/len(pi_data)*100),
      "% ---")
# for each FASTA file
count_done = 0
total_start_time = time.time()
for i, row in pi_passed_data.iterrows():
    EC = row['EC.code'].replace(" ", "")
    EC_ = EC.replace(".", "_")
    # get brenda data file
    br_datafile = 'brenda_download_'+EC_+'.txt'
    if os.path.isfile(br_datafile) is False:
        src = param_dict['BRENDA_DB_loc']+br_datafile
        des = database_directory+br_datafile
        copyfile(src, des)
    print(EC)
    print(br_datafile)
    print(row)
    # convert br_datafile into dictionary
    br_symbols, br_data = brenda_IO.get_brenda_dict(database_directory+br_datafile)
    # collect protein assocoiated with species of FASTA file
    
    
    break
    
print('--- finished %s proteins in %s seconds ---' % (count_done, '{0:.2f}'.format(time.time() - total_start_time)))
print('=====================================================================')
print('                              Complete!                              ')
print('=====================================================================')

                     Screening enzymes with Brenda!                  
---  2440 proteins out of 2750 remain, OR 88 % ---
6.6.1.2
brenda_download_6_6_1_2.txt
fasta_file      /home/atarzia/psp/sequence_db/test_dataset/6_6...
acc.code                                                  P29933 
organism                                        cobaltochelatase 
EC.code                                                  6.6.1.2 
species                                Pseudomonas denitrificans 
note                                                   Swiss-Prot
pi                                                           5.69
modification                                                    0
category                                                        0
Name: 0, dtype: object
--- finished 0 proteins in 0.00 seconds ---
                              Complete!                              


In [13]:
br_data['PR']

['#1# Pseudomonas denitrificans (nomen rejiciendum)   <1>',
 '#2# Salmonella enterica   <4>',
 '#3# Desulfovibrio vulgaris   <4>',
 '#4# Brucella melitensis   <3>',
 '#5# Pseudomonas denitrificans (nomen rejiciendum) P29933 AND P29934 ANDQ9HZQ3 AND P29929']

In [14]:
for prot in br_data['PR']:
    if row['species'] in prot:
        print(prot)

#1# Pseudomonas denitrificans (nomen rejiciendum)   <1>
#5# Pseudomonas denitrificans (nomen rejiciendum) P29933 AND P29934 ANDQ9HZQ3 AND P29929


In [15]:
pi_data[pi_data['EC.code'] == ' 4.5.1.1 ']

,fasta_file,acc.code,organism,EC.code,species,note,pi,modification,category
